# Data Cleaning + Preparing
team member: Sonia

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime as dt
import csv

In [4]:
# go back to grand parent level directory
path = os.path.abspath(os.getcwd() + "/../../")

1.	How could they improve their pricing strategy?
2.	What happened when the price was increased?
3.	What does this suggest as possible options going forward?
4.	In addition to price, what other factors influence whether a member is going to leave?
5.	Bonus: Given this, to what degree can we predict individual member churn?

## Import Raw and processed data

-----------------------------
### Import client and competitor pricing data

In [3]:
# import client data
client_price = pd.read_csv(os.path.join(path,'data/processed/client_price.csv'))
client_price

Year  Flexible  Standard  Joinee  Dayass
0  2009     25.99     20.99       5       6
1  2010     25.99     20.99       5       6
2  2011     25.99     20.99       5       6
3  2012     25.99     20.99       5       6
4  2013     25.99     20.99       5       6
5  2014     25.99     20.99       5       6
6  2015     25.99     20.99       5       6
7  2016     30.99     26.99       5       6
8  2017     30.99     26.99       5       6
9  2018     30.99     26.99       5       6

In [4]:
# import competitor data
competitor_price = pd.read_csv(os.path.join(path,'data/processed/competitor_price.csv'))
competitor_price.head(2)

Year  Flexible  Standard  Joinee
0  2009     35.99     30.99       8
1  2010     35.99     30.99       8


------------------------
### Create a customer visit database using visitationdataPART1.csv and visitationdataPART2.csv

In [5]:
# import visit data

def visit():
    visit_1 = pd.read_csv(os.path.join(path,'data/raw/visitationdataPART1.csv'))
    visit_2 = pd.read_csv(os.path.join(path,'data/raw/visitationdataPART2.csv'))
    visit_db = pd.concat([visit_1, visit_2])
    
    # combine columns to create date + time onject
    visit_db['timestamp'] = visit_db['visitdate']+' '+visit_db['TimeofVisit']
    visit_db['timestamp'] = pd.to_datetime(visit_db['timestamp'])
    visit_db['is_peak'] = [1 if x else 0 for x in visit_db['peakvisits']]

    visit_db = visit_db.drop(columns=['peakvisits', 'visitdate', 'TimeofVisit'])
    visit_db = visit_db.rename(columns={"CustomerID": "id"})
    
    print(f'{len(visit_db)} visits found and imported')
    return visit_db

In [6]:
visit_db = visit()
visit_db.head(2)

69514576 visits found and imported


id           timestamp  is_peak
0  126444 2009-05-01 05:45:00        0
1  456610 2009-05-01 05:45:00        0

In [7]:
# round up to next day to find the cutoff time
from pandas.tseries.offsets import MonthEnd
as_of_date = visit_db['timestamp'].max().round('D')+ MonthEnd(1)
as_of_date

Timestamp('2019-01-31 00:00:00')


------------------------
###  Create a customer subscription database using subcriptiondataFINAL.csv

In [8]:
def membership():
    #### membership database ####
    # import subscription data and create a membership database
    membership_list = []
        
    with open(os.path.join(path,'data/raw/subscriptionsdataFINAL.csv')) as subscription_file:    
        read_csv = csv.reader(subscription_file, delimiter=',')  
        next(read_csv, None)    # skip the headers
                
        # a pct score based on % of total population ranked below a certain class.
        # information source: National Readership Survey
        # http://www.nrs.co.uk/nrs-print/lifestyle-and-classification-data/social-grade/
        affluence_rank = {'A' : 0.96,     # upper middle
                           'B' : 0.73,    # middle
                           'C1': 0.45,    # lower middle
                           'C2': 0.25,    # skilled working
                           'DE': 0}       # lower working and non-working
        
        for r in read_csv:
            club = r[0]
            join_mth = dt.datetime.strptime(r[1], '%b-%y')
            s_cls = affluence_rank[r[2]]
            is_male = (1 if r[3] == 'Male' else 0)
            is_std = (1 if r[4] == 'Standard' else 0)                     
            id = int(r[6])

            # calculate age at joining the gym. add extra year if joining in July or later.
            join_age = join_mth.year - int(r[7]) + (1 if join_mth.month >= 7 else 0)
            
            # fix end_dates
            if r[5] != '#N/A':
                end_mth = dt.datetime.strptime(r[5], '%b-%y')
                is_active = 0
                end_date = end_mth + dt.timedelta(days=31)    # calculate <1 month as 1 month 
                
                ######## check whether left due to price hike #####
                is_left_Dec15 = 1 if end_mth == dt.datetime(2015, 12 ,1) else 0

            else:
                end_mth = np.nan
                is_active = 1
                end_date = as_of_date + dt.timedelta(days=1)    # calculate last date as of 1-Feb
                is_left_Dec15 = 0
                
            ######## MORE summary data ########
            # membership duration in months
            m_dur = (end_date.year - join_mth.year) * 12 + end_date.month - join_mth.month
            
            # member revenue to 2015
            if join_mth < dt.datetime(2015,12,31):
                cutoff = min(end_date, dt.datetime(2016,1,1))
                m_to_15 = (cutoff.year - join_mth.year) * 12 + cutoff.month - join_mth.month
            else:
                m_to_15 = 0
                
            rev_to_15 = m_to_15 * (20.99 if is_std == 1 else 25.99)
            
            # member revenue fr 2016
            m_fr_16 = m_dur if join_mth > dt.datetime(2015,12,31) else 0  
            rev_fr_16 = m_fr_16 * (26.99 if is_std == 1 else 30.99)
            
            # concat all member information
            member_info = [id, is_male, join_age, s_cls, club, join_mth, end_mth, is_std, 
                            m_dur, is_left_Dec15, is_active, end_date, rev_to_15, rev_fr_16, rev_to_15 + rev_fr_16,]
            membership_list.append(member_info)
    
    column_name = ['id', 'is_male', 'join_age', 's_class_rank', 'club', 'join_mth', 'end_mth','is_std',
                   'mth_active', 'is_left_Dec15', 'is_active', 'last_active_date', 'rev_to_15', 'rev_fr_16', 'rev_to_date']
    
    membership_db = pd.DataFrame(membership_list, columns=column_name)
    
    print(f'{len(membership_db)} members found and imported')
    return membership_db

In [9]:
membership_db = membership()
membership_db.head(2)

850033 members found and imported


id  is_male  join_age  s_class_rank       club   join_mth    end_mth  \
0   1        1        20          0.45    Enfield 2011-12-01 2011-12-01   
1   2        0        46          0.96  Greenwich 2018-06-01        NaT   

   is_std  mth_active  is_left_Dec15  is_active last_active_date  rev_to_15  \
0       1           1              0          0       2012-01-01      20.99   
1       1           8              0          1       2019-02-01       0.00   

   rev_fr_16  rev_to_date  
0       0.00        20.99  
1     215.92       215.92


------------------------
### Summerise visit data and add useful attributes to membership database

#### Create some summary stats by customer_id
- first visit
- last visit
- total visit
- total peak hour visit


In [10]:
first_visit = visit_db.groupby('id').nth(0)['timestamp'].rename('first_visit').to_frame().reset_index()
last_visit = visit_db.groupby('id').nth(-1)['timestamp'].rename('last_visit').to_frame().reset_index()
total_visit = visit_db.groupby('id')['timestamp'].count().rename('total_visit').to_frame().reset_index()
total_peak_hour_visit = visit_db.groupby('id')['is_peak'].sum().rename('total_peak').to_frame().reset_index()

In [11]:
# merge data onto membership_db
membership_db = membership_db.merge(first_visit, how='left', left_on=['id'], right_on = ['id'])
membership_db = membership_db.merge(last_visit, how='left', left_on=['id'], right_on = ['id'])
membership_db = membership_db.merge(total_visit, how='left', left_on=['id'], right_on = ['id'])
membership_db = membership_db.merge(total_peak_hour_visit, how='left', left_on=['id'], right_on = ['id'])

In [12]:
# fill in nans
membership_db['total_visit'] = membership_db['total_visit'].fillna(0).astype(int)
membership_db['total_peak'] = membership_db['total_peak'].fillna(0).astype(int)

days = membership_db['last_active_date'] - membership_db['join_mth']
membership_db['pct_gym_usage'] = membership_db['total_visit'].divide(days.dt.days)
membership_db['pct_peak'] = membership_db['total_peak'].divide(membership_db['total_visit'])

membership_db.loc[~np.isfinite(membership_db['pct_gym_usage']), 'pct_gym_usage'] = 0
membership_db.loc[~np.isfinite(membership_db['pct_peak']), 'pct_peak'] = 0

In [13]:
membership_db = membership_db.drop(columns=['last_active_date'])
membership_db.head(2)

id  is_male  join_age  s_class_rank       club   join_mth    end_mth  \
0   1        1        20          0.45    Enfield 2011-12-01 2011-12-01   
1   2        0        46          0.96  Greenwich 2018-06-01        NaT   

   is_std  mth_active  is_left_Dec15  is_active  rev_to_15  rev_fr_16  \
0       1           1              0          0      20.99       0.00   
1       1           8              0          1       0.00     215.92   

   rev_to_date         first_visit          last_visit  total_visit  \
0        20.99 2011-12-01 08:00:00 2011-12-27 18:45:00           20   
1       215.92 2018-06-01 17:30:00 2019-01-08 18:30:00           74   

   total_peak  pct_gym_usage  pct_peak  
0           9       0.645161  0.450000  
1          41       0.302041  0.554054

### Create some summary stats by gym
- gym total revenue up to dec 2015
- gym total revenue from Jan 2016
- Number of people left in Dec 2015 before price rise
- gym's revenue contribution

In [127]:
club_rev = membership_db.groupby('club').sum()[['rev_to_15', 'rev_fr_16', 'rev_to_date']].sort_values(by=['rev_to_date'], ascending=False)
club_rev['rev_%'] = club_rev['rev_to_date']/club_rev['rev_to_date'].sum()

member_loss_dec15 = membership_db.groupby('club')['is_left_Dec15'].sum().rename('member_left_Dec15').to_frame().reset_index()
member_loss_all = membership_db.groupby('club')['is_left_Dec15'].sum().rename('member_left').to_frame().reset_index()
member_loss_all = membership_db.groupby('club')['is_left_Dec15'].sum().rename('member_left_Dec15').to_frame().reset_index()
club_rev = club_rev.merge(member_loss_dec15, how='left', left_on=['club'], right_on = ['club'])

#club_rev['loss_%'] = club_rev['rev_to_date']/club_rev['rev_to_date'].sum()
club_rev.head()


club     rev_to_15   rev_fr_16   rev_to_date     rev_%  \
0              Hounslow  3.745534e+06  1976655.44  5.722190e+06  0.056200   
1             Greenwich  3.558127e+06  1881685.01  5.439812e+06  0.053427   
2              Havering  2.959284e+06  2359068.66  5.318353e+06  0.052234   
3  Richmond upon Thames  3.358700e+06  1658391.26  5.017091e+06  0.049275   
4                Harrow  2.908353e+06  1949868.69  4.858221e+06  0.047715   

   member_left_Dec15  
0                873  
1                794  
2                980  
3                756  
4                845

### Club environment 

In [90]:
mthly_visits = visit_db.groupby([visit_db.timestamp, visit_db.id]).count()
mthly_visits.head()

is_peak
timestamp           id             
2009-05-01 00:00:00 377193        1
2009-05-01 05:45:00 126444        1
                    456610        1
2009-05-01 06:00:00 293763        1
2009-05-01 06:15:00 151253        1

In [99]:
mthly_visits_1 = mthly_visits.reset_index()
mthly_visits_1['year_mth'] = [dt.datetime(t.year, t.month, 1) for t in mthly_visits_1['timestamp']]


In [100]:
mthly_visits_1.head()

timestamp      id  is_peak   year_mth
0 2009-05-01 00:00:00  377193        1 2009-05-01
1 2009-05-01 05:45:00  126444        1 2009-05-01
2 2009-05-01 05:45:00  456610        1 2009-05-01
3 2009-05-01 06:00:00  293763        1 2009-05-01
4 2009-05-01 06:15:00  151253        1 2009-05-01

In [101]:
mthly_visits_2 = mthly_visits_1[['id','is_peak','year_mth']].merge(membership_db[['id','club']], how='left', left_on=['id'], right_on = ['id'])

In [102]:
mthly_visits_2.head()

id  is_peak   year_mth      club
0  377193        1 2009-05-01  Hounslow
1  126444        1 2009-05-01  Hounslow
2  456610        1 2009-05-01  Hounslow
3  293763        1 2009-05-01  Hounslow
4  151253        1 2009-05-01  Hounslow

#### calculate avg monthly active member

In [120]:
gym_cap = mthly_visits_2.groupby(['club','year_mth'])['id'].nunique().rename('active_member').to_frame().reset_index()
gym_cap = gym_cap.groupby(['club'])['active_member'].mean().rename('avg_mthly_active_member').to_frame().reset_index()
gym_cap.head()

club  avg_mthly_active_member
0  Barking              2359.600000
1   Barnet              2017.927083
2   Bexley              1909.209302
3    Brent              1977.230769
4  Bromley              2263.857143

#### calculate avg monthly visits

In [126]:
gym_visit = mthly_visits_2.groupby(['club','year_mth'])['is_peak'].count().rename('visits').to_frame().reset_index()
gym_visit = gym_visit.groupby(['club'])['visits'].mean().rename('gym_mthly_visit').to_frame().reset_index()
gym_visit.head()

club  gym_mthly_visit
0  Barking     38665.400000
1   Barnet     33058.270833
2   Bexley     31167.837209
3    Brent     31451.442308
4  Bromley     37109.892857

In [128]:
club_rev = club_rev.merge(gym_cap, how='left', left_on=['club'], right_on = ['club'])
club_rev = club_rev.merge(gym_visit, how='left', left_on=['club'], right_on = ['club'])
club_rev.head()

club     rev_to_15   rev_fr_16   rev_to_date     rev_%  \
0              Hounslow  3.745534e+06  1976655.44  5.722190e+06  0.056200   
1             Greenwich  3.558127e+06  1881685.01  5.439812e+06  0.053427   
2              Havering  2.959284e+06  2359068.66  5.318353e+06  0.052234   
3  Richmond upon Thames  3.358700e+06  1658391.26  5.017091e+06  0.049275   
4                Harrow  2.908353e+06  1949868.69  4.858221e+06  0.047715   

   member_left_Dec15  avg_mthly_active_member  gym_mthly_visit  
0                873              2125.581197     34880.376068  
1                794              2056.913043     33372.886957  
2                980              2484.826087     41226.510870  
3                756              1961.142857     32336.883929  
4                845              2172.525773     36388.000000

#### attached to membership

In [134]:
membership_db = membership_db.merge(club_rev[['club','rev_%','gym_mthly_visit']], how='left', left_on=['club'], right_on = ['club'])
membership_db['is_age_below_40'] = [1 if t <= 40 else 0 for t in membership_db['join_age']]
membership_db['is_age_above_40'] = [1 if t > 40 else 0 for t in membership_db['join_age']]
membership_db.head(2)

id  is_male  join_age  s_class_rank       club   join_mth    end_mth  \
0   1        1        20          0.45    Enfield 2011-12-01 2011-12-01   
1   2        0        46          0.96  Greenwich 2018-06-01        NaT   

   is_std  mth_active  is_left_Dec15       ...         total_visit  \
0       1           1              0       ...                  20   
1       1           8              0       ...                  74   

   total_peak  pct_gym_usage  pct_peak   rev_%_x gym_mthly_visit_x  \
0           9       0.645161  0.450000  0.040164      27020.523364   
1          41       0.302041  0.554054  0.053427      33372.886957   

   is_age_below_40   rev_%_y  gym_mthly_visit_y  is_age_above_40  
0                1  0.040164       27020.523364                0  
1                0  0.053427       33372.886957                1  

[2 rows x 26 columns]

_____________________

### Export to CSV

In [133]:
membership_db.to_csv(os.path.join(path,'data/processed/membership_clean.csv'), index=False)   
membership_db.head(2)

id  is_male  join_age  s_class_rank       club   join_mth    end_mth  \
0   1        1        20          0.45    Enfield 2011-12-01 2011-12-01   
1   2        0        46          0.96  Greenwich 2018-06-01        NaT   

   is_std  mth_active  is_left_Dec15       ...         rev_to_date  \
0       1           1              0       ...               20.99   
1       1           8              0       ...              215.92   

          first_visit          last_visit  total_visit total_peak  \
0 2011-12-01 08:00:00 2011-12-27 18:45:00           20          9   
1 2018-06-01 17:30:00 2019-01-08 18:30:00           74         41   

  pct_gym_usage  pct_peak     rev_%  gym_mthly_visit  is_age_below_40  
0      0.645161  0.450000  0.040164     27020.523364                1  
1      0.302041  0.554054  0.053427     33372.886957                0  

[2 rows x 23 columns]

In [77]:
"""
visit_db.to_csv(os.path.join(path,'data/processed/visit_clean.csv'), index=False)

first_visit.to_csv(os.path.join(path,'data/processed/member_first_visit.csv'), index=False)
last_visit.to_csv(os.path.join(path,'data/processed/member_last_visit.csv'), index=False)
total_visit.to_csv(os.path.join(path,'data/processed/member_total_visit.csv'), index=False)
total_peak_hour_visit.to_csv(os.path.join(path,'data/processed/member_total_peak_visit.csv'), index=False)

membership_db.to_csv(os.path.join(path,'data/processed/membership_clean.csv'), index=False)   
club_rev.to_csv(os.path.join(path,'data/processed/gym_profit.csv'), index=False)
"""